# 🔬 Exercícios Práticos: Teste de Hipóteses

Este notebook contém exercícios práticos sobre testes de hipóteses estatísticos, incluindo testes t, qui-quadrado e ANOVA.

## Referências Teóricas

Os exercícios são baseados em:
- **Montgomery, D.C.; Runger, G.C. (2018).** *Applied Statistics and Probability for Engineers*. 7. ed. Hoboken: Wiley.
- **Field, A. (2018).** *Discovering Statistics Using IBM SPSS Statistics*. 5. ed. London: SAGE.
- **Cohen, J. (1988).** *Statistical Power Analysis for the Behavioral Sciences*. 2. ed. Hillsdale: LEA.

In [ ]:
# Bibliotecas necessárias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, t as t_dist, chi2, f as f_dist
import warnings
warnings.filterwarnings('ignore')

# Configurações
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
np.random.seed(42)

## Exercício 1: Teste t para Uma Amostra

### Problema
Uma fábrica afirma que seus parafusos têm diâmetro médio de 8mm. Um inspetor coleta 25 parafusos e mede os diâmetros. Testar se a média populacional é realmente 8mm.

### Fundamento Teórico
O **teste t de Student** (Gosset, 1908) é usado quando:
- A variância populacional é desconhecida
- A amostra é pequena (n < 30) ou população é normal

Estatística de teste:
$$t = \frac{\bar{x} - \mu_0}{s / \sqrt{n}}$$

onde $\bar{x}$ é a média amostral, $\mu_0$ é o valor hipotético, $s$ é o desvio padrão amostral, e $n$ é o tamanho da amostra.

**Referência:** Montgomery & Runger (2018), Capítulo 9.

In [ ]:
# Solução do Exercício 1

# Dados simulados
np.random.seed(123)
n = 25
mu_real = 8.05  # Média real (ligeiramente diferente do especificado)
diametros = np.random.normal(mu_real, 0.15, n)

# Hipóteses
mu_0 = 8.0  # Valor especificado
alpha = 0.05

print("="*70)
print("TESTE T PARA UMA AMOSTRA")
print("="*70)
print(f"\nH₀: μ = {mu_0}mm (média é {mu_0}mm como especificado)")
print(f"H₁: μ ≠ {mu_0}mm (média é diferente de {mu_0}mm)")
print(f"\nNível de significância: α = {alpha}")

# Estatísticas descritivas
x_bar = diametros.mean()
s = diametros.std(ddof=1)
se = s / np.sqrt(n)

print(f"\n--- Dados da Amostra ---")
print(f"Tamanho (n): {n}")
print(f"Média amostral (x̄): {x_bar:.4f} mm")
print(f"Desvio padrão (s): {s:.4f} mm")
print(f"Erro padrão (SE): {se:.4f} mm")

# Calcular estatística t
t_calculado = (x_bar - mu_0) / se
gl = n - 1  # graus de liberdade

# Valor crítico e p-valor
t_critico = t_dist.ppf(1 - alpha/2, gl)
p_valor = 2 * (1 - t_dist.cdf(abs(t_calculado), gl))

print(f"\n--- Teste de Hipótese ---")
print(f"Estatística t: {t_calculado:.4f}")
print(f"Graus de liberdade: {gl}")
print(f"Valor crítico (bilateral): ±{t_critico:.4f}")
print(f"Valor-p: {p_valor:.4f}")

# Decisão
print(f"\n--- Decisão ---")
if p_valor < alpha:
    print(f"✗ p-valor ({p_valor:.4f}) < α ({alpha})")
    print(f"   REJEITAMOS H₀")
    print(f"   Conclusão: Há evidência significativa de que a média é diferente de {mu_0}mm")
else:
    print(f"✓ p-valor ({p_valor:.4f}) ≥ α ({alpha})")
    print(f"   NÃO REJEITAMOS H₀")
    print(f"   Conclusão: Não há evidência suficiente para dizer que a média difere de {mu_0}mm")

# Intervalo de confiança
ic_lower = x_bar - t_critico * se
ic_upper = x_bar + t_critico * se
print(f"\nIntervalo de Confiança 95%: [{ic_lower:.4f}, {ic_upper:.4f}]")

# Tamanho do efeito (d de Cohen)
d_cohen = (x_bar - mu_0) / s
print(f"\nTamanho do efeito (d de Cohen): {d_cohen:.4f}")
if abs(d_cohen) < 0.2:
    tamanho = "pequeno"
elif abs(d_cohen) < 0.5:
    tamanho = "médio"
else:
    tamanho = "grande"
print(f"Interpretação: Efeito {tamanho} (Cohen, 1988)")

# Verificar com scipy
t_stat_scipy, p_valor_scipy = stats.ttest_1samp(diametros, mu_0)
print(f"\n--- Verificação com scipy.stats ---")
print(f"t = {t_stat_scipy:.4f}, p = {p_valor_scipy:.4f} ✓")

# Visualização
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico 1: Distribuição dos dados
axes[0].hist(diametros, bins=10, alpha=0.7, color='skyblue', edgecolor='black')
axes[0].axvline(x_bar, color='blue', linestyle='--', linewidth=2, label=f'Média amostral: {x_bar:.3f}')
axes[0].axvline(mu_0, color='red', linestyle='--', linewidth=2, label=f'Valor hipotético: {mu_0}')
axes[0].set_xlabel('Diâmetro (mm)', fontsize=11)
axes[0].set_ylabel('Frequência', fontsize=11)
axes[0].set_title('Distribuição dos Diâmetros', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Gráfico 2: Distribuição t e região de rejeição
x = np.linspace(-4, 4, 1000)
y = t_dist.pdf(x, gl)
axes[1].plot(x, y, 'b-', linewidth=2, label='Distribuição t')
axes[1].axvline(t_calculado, color='red', linestyle='--', linewidth=2, label=f't calculado: {t_calculado:.3f}')
axes[1].axvline(-t_critico, color='green', linestyle=':', linewidth=1.5, label=f't crítico: ±{t_critico:.3f}')
axes[1].axvline(t_critico, color='green', linestyle=':', linewidth=1.5)

# Região de rejeição
x_left = x[x < -t_critico]
x_right = x[x > t_critico]
axes[1].fill_between(x_left, t_dist.pdf(x_left, gl), alpha=0.3, color='red', label='Região de rejeição')
axes[1].fill_between(x_right, t_dist.pdf(x_right, gl), alpha=0.3, color='red')

axes[1].set_xlabel('Estatística t', fontsize=11)
axes[1].set_ylabel('Densidade', fontsize=11)
axes[1].set_title(f'Teste t (gl={gl}, α={alpha})', fontsize=12, fontweight='bold')
axes[1].legend(fontsize=9)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Exercício 2: Teste t para Duas Amostras Independentes

### Problema
Comparar o desempenho médio de dois grupos de estudantes: um que recebeu tutoria online e outro que teve aulas presenciais.

### Fundamento Teórico
O **teste t para amostras independentes** compara as médias de dois grupos não relacionados.

Estatística de teste (variâncias iguais):
$$t = \frac{\bar{x}_1 - \bar{x}_2}{s_p \sqrt{\frac{1}{n_1} + \frac{1}{n_2}}}$$

onde $s_p$ é o desvio padrão combinado (pooled).

**Referência:** Field (2018), Capítulo 10.

In [ ]:
# Solução do Exercício 2

# Dados simulados
np.random.seed(456)
grupo_online = np.random.normal(75, 10, 30)
grupo_presencial = np.random.normal(80, 10, 30)

# Hipóteses
alpha = 0.05

print("="*70)
print("TESTE T PARA DUAS AMOSTRAS INDEPENDENTES")
print("="*70)
print(f"\nH₀: μ₁ = μ₂ (médias são iguais)")
print(f"H₁: μ₁ ≠ μ₂ (médias são diferentes)")
print(f"\nNível de significância: α = {alpha}")

# Estatísticas descritivas
print(f"\n--- Estatísticas Descritivas ---")
print(f"\nGrupo Online:")
print(f"  n₁ = {len(grupo_online)}")
print(f"  x̄₁ = {grupo_online.mean():.2f}")
print(f"  s₁ = {grupo_online.std(ddof=1):.2f}")

print(f"\nGrupo Presencial:")
print(f"  n₂ = {len(grupo_presencial)}")
print(f"  x̄₂ = {grupo_presencial.mean():.2f}")
print(f"  s₂ = {grupo_presencial.std(ddof=1):.2f}")

# Teste de Levene para homogeneidade de variâncias
stat_levene, p_levene = stats.levene(grupo_online, grupo_presencial)
print(f"\n--- Teste de Levene (Homogeneidade de Variâncias) ---")
print(f"Estatística: {stat_levene:.4f}")
print(f"p-valor: {p_levene:.4f}")

if p_levene > 0.05:
    print("✓ Variâncias homogêneas (usar teste t padrão)")
    equal_var = True
else:
    print("✗ Variâncias não homogêneas (usar correção de Welch)")
    equal_var = False

# Teste t
t_stat, p_valor = stats.ttest_ind(grupo_online, grupo_presencial, equal_var=equal_var)

print(f"\n--- Teste t para Amostras Independentes ---")
print(f"Estatística t: {t_stat:.4f}")
print(f"Valor-p: {p_valor:.4f}")

# Decisão
print(f"\n--- Decisão ---")
if p_valor < alpha:
    print(f"✗ p-valor ({p_valor:.4f}) < α ({alpha})")
    print(f"   REJEITAMOS H₀")
    print(f"   Há diferença significativa entre os grupos")
else:
    print(f"✓ p-valor ({p_valor:.4f}) ≥ α ({alpha})")
    print(f"   NÃO REJEITAMOS H₀")
    print(f"   Não há evidência de diferença significativa")

# Tamanho do efeito (d de Cohen)
mean_diff = grupo_presencial.mean() - grupo_online.mean()
pooled_std = np.sqrt(((len(grupo_online)-1)*grupo_online.var(ddof=1) + 
                       (len(grupo_presencial)-1)*grupo_presencial.var(ddof=1)) / 
                      (len(grupo_online) + len(grupo_presencial) - 2))
d_cohen = mean_diff / pooled_std

print(f"\nDiferença de médias: {mean_diff:.2f}")
print(f"d de Cohen: {d_cohen:.4f}")
if abs(d_cohen) < 0.2:
    print("Tamanho do efeito: pequeno")
elif abs(d_cohen) < 0.5:
    print("Tamanho do efeito: médio")
elif abs(d_cohen) < 0.8:
    print("Tamanho do efeito: médio-grande")
else:
    print("Tamanho do efeito: grande")

# Visualização
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico 1: Boxplot
data_plot = pd.DataFrame({
    'Nota': np.concatenate([grupo_online, grupo_presencial]),
    'Grupo': ['Online']*len(grupo_online) + ['Presencial']*len(grupo_presencial)
})

sns.boxplot(x='Grupo', y='Nota', data=data_plot, ax=axes[0], palette='Set2')
sns.swarmplot(x='Grupo', y='Nota', data=data_plot, ax=axes[0], color='black', alpha=0.3, size=3)
axes[0].set_title('Comparação de Desempenho', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Nota', fontsize=11)
axes[0].grid(True, alpha=0.3, axis='y')

# Gráfico 2: Distribuições
axes[1].hist(grupo_online, bins=15, alpha=0.5, label='Online', color='skyblue', edgecolor='black')
axes[1].hist(grupo_presencial, bins=15, alpha=0.5, label='Presencial', color='lightcoral', edgecolor='black')
axes[1].axvline(grupo_online.mean(), color='blue', linestyle='--', linewidth=2)
axes[1].axvline(grupo_presencial.mean(), color='red', linestyle='--', linewidth=2)
axes[1].set_xlabel('Nota', fontsize=11)
axes[1].set_ylabel('Frequência', fontsize=11)
axes[1].set_title('Distribuição das Notas', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Exercício 3: Teste Qui-Quadrado de Independência

### Problema
Investigar se há associação entre gênero e preferência por tipo de filme (ação, romance, comédia).

### Fundamento Teórico
O **teste qui-quadrado** (χ²) de Pearson testa independência entre variáveis categóricas:

$$\chi^2 = \sum_{i,j} \frac{(O_{ij} - E_{ij})^2}{E_{ij}}$$

onde $O_{ij}$ são frequências observadas e $E_{ij}$ são frequências esperadas sob independência.

**Referência:** Field (2018), Capítulo 18.

In [ ]:
# Solução do Exercício 3

# Dados: Tabela de contingência
dados = pd.DataFrame({
    'Ação': [45, 25],
    'Romance': [20, 50],
    'Comédia': [35, 25]
}, index=['Masculino', 'Feminino'])

print("="*70)
print("TESTE QUI-QUADRADO DE INDEPENDÊNCIA")
print("="*70)
print(f"\nH₀: Gênero e preferência de filme são independentes")
print(f"H₁: Gênero e preferência de filme são associados")
print(f"\nNível de significância: α = 0.05")

# Frequências observadas
print("\n--- Frequências Observadas ---")
print(dados)
print(f"\nTotal: {dados.values.sum()}")

# Teste qui-quadrado
chi2_stat, p_valor, gl, freq_esperadas = stats.chi2_contingency(dados)

# Frequências esperadas
print("\n--- Frequências Esperadas (sob H₀) ---")
df_esperadas = pd.DataFrame(freq_esperadas, 
                             index=dados.index, 
                             columns=dados.columns)
print(df_esperadas.round(2))

# Resultados do teste
print(f"\n--- Teste Qui-Quadrado ---")
print(f"Estatística χ²: {chi2_stat:.4f}")
print(f"Graus de liberdade: {gl}")
print(f"Valor-p: {p_valor:.4f}")

# Valor crítico
alpha = 0.05
chi2_critico = chi2.ppf(1-alpha, gl)
print(f"Valor crítico (α=0.05): {chi2_critico:.4f}")

# Decisão
print(f"\n--- Decisão ---")
if p_valor < alpha:
    print(f"✗ p-valor ({p_valor:.4f}) < α ({alpha})")
    print(f"   REJEITAMOS H₀")
    print(f"   Há associação significativa entre gênero e preferência")
else:
    print(f"✓ p-valor ({p_valor:.4f}) ≥ α ({alpha})")
    print(f"   NÃO REJEITAMOS H₀")
    print(f"   Não há evidência de associação")

# Resíduos padronizados
residuos = (dados.values - freq_esperadas) / np.sqrt(freq_esperadas)
df_residuos = pd.DataFrame(residuos, 
                            index=dados.index, 
                            columns=dados.columns)

print("\n--- Resíduos Padronizados ---")
print(df_residuos.round(2))
print("\nInterpretação: |resíduo| > 2 indica contribuição forte para χ²")

# Visualização
fig, axes = plt.subplots(1, 3, figsize=(16, 4))

# Gráfico 1: Frequências observadas
dados.plot(kind='bar', ax=axes[0], color=['#FF6B6B', '#4ECDC4', '#FFE66D'], edgecolor='black')
axes[0].set_title('Frequências Observadas', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Gênero', fontsize=11)
axes[0].set_ylabel('Frequência', fontsize=11)
axes[0].legend(title='Tipo de Filme', fontsize=9)
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=0)
axes[0].grid(True, alpha=0.3, axis='y')

# Gráfico 2: Frequências esperadas
df_esperadas.plot(kind='bar', ax=axes[1], color=['#FF6B6B', '#4ECDC4', '#FFE66D'], 
                   alpha=0.7, edgecolor='black')
axes[1].set_title('Frequências Esperadas (sob H₀)', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Gênero', fontsize=11)
axes[1].set_ylabel('Frequência', fontsize=11)
axes[1].legend(title='Tipo de Filme', fontsize=9)
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=0)
axes[1].grid(True, alpha=0.3, axis='y')

# Gráfico 3: Heatmap de resíduos
sns.heatmap(df_residuos, annot=True, fmt='.2f', cmap='RdBu_r', center=0, 
             cbar_kws={'label': 'Resíduo Padronizado'}, ax=axes[2],
             vmin=-3, vmax=3, linewidths=1, linecolor='black')
axes[2].set_title('Resíduos Padronizados', fontsize=12, fontweight='bold')
axes[2].set_xlabel('Tipo de Filme', fontsize=11)
axes[2].set_ylabel('Gênero', fontsize=11)

plt.tight_layout()
plt.show()

# Tamanho do efeito (V de Cramér)
n_total = dados.values.sum()
min_dim = min(dados.shape[0] - 1, dados.shape[1] - 1)
cramer_v = np.sqrt(chi2_stat / (n_total * min_dim))
print(f"\nV de Cramér: {cramer_v:.4f}")
if cramer_v < 0.1:
    print("Tamanho do efeito: muito pequeno")
elif cramer_v < 0.3:
    print("Tamanho do efeito: pequeno")
elif cramer_v < 0.5:
    print("Tamanho do efeito: médio")
else:
    print("Tamanho do efeito: grande")

## Exercício 4: ANOVA (Análise de Variância)

### Problema
Comparar o efeito de três diferentes fertilizantes no crescimento de plantas.

### Fundamento Teórico
A **ANOVA** (Fisher, 1925) testa se há diferença entre médias de três ou mais grupos:

$$F = \frac{MS_{between}}{MS_{within}} = \frac{SS_{between}/(k-1)}{SS_{within}/(N-k)}$$

onde $k$ é o número de grupos e $N$ é o tamanho total da amostra.

**Referência:** Montgomery & Runger (2018), Capítulo 13.

In [ ]:
# Solução do Exercício 4

# Dados simulados: Crescimento de plantas (cm) com 3 fertilizantes
np.random.seed(789)
fertilizante_A = np.random.normal(20, 3, 15)
fertilizante_B = np.random.normal(24, 3, 15)
fertilizante_C = np.random.normal(22, 3, 15)

# Criar DataFrame
df = pd.DataFrame({
    'Crescimento': np.concatenate([fertilizante_A, fertilizante_B, fertilizante_C]),
    'Fertilizante': ['A']*15 + ['B']*15 + ['C']*15
})

print("="*70)
print("ANOVA - ANÁLISE DE VARIÂNCIA")
print("="*70)
print(f"\nH₀: μ_A = μ_B = μ_C (todas as médias são iguais)")
print(f"H₁: Pelo menos uma média é diferente")
print(f"\nNível de significância: α = 0.05")

# Estatísticas descritivas
print("\n--- Estatísticas Descritivas ---")
print(df.groupby('Fertilizante')['Crescimento'].describe().round(2))

# ANOVA
grupos = [fertilizante_A, fertilizante_B, fertilizante_C]
f_stat, p_valor = stats.f_oneway(*grupos)

print(f"\n--- ANOVA ---")
print(f"Estatística F: {f_stat:.4f}")
print(f"Valor-p: {p_valor:.4f}")

# Decisão
alpha = 0.05
print(f"\n--- Decisão ---")
if p_valor < alpha:
    print(f"✗ p-valor ({p_valor:.4f}) < α ({alpha})")
    print(f"   REJEITAMOS H₀")
    print(f"   Há diferença significativa entre os fertilizantes")
else:
    print(f"✓ p-valor ({p_valor:.4f}) ≥ α ({alpha})")
    print(f"   NÃO REJEITAMOS H₀")
    print(f"   Não há evidência de diferença entre os fertilizantes")

# Tamanho do efeito (eta²)
grand_mean = df['Crescimento'].mean()
ss_total = ((df['Crescimento'] - grand_mean) ** 2).sum()
ss_between = sum([len(g) * (g.mean() - grand_mean)**2 for g in grupos])
eta_squared = ss_between / ss_total

print(f"\nEta² (η²): {eta_squared:.4f}")
if eta_squared < 0.01:
    print("Tamanho do efeito: pequeno")
elif eta_squared < 0.06:
    print("Tamanho do efeito: médio")
else:
    print("Tamanho do efeito: grande")

# Testes post-hoc (se ANOVA significativa)
if p_valor < alpha:
    print("\n--- Testes Post-Hoc (Tukey HSD) ---")
    from scipy.stats import tukey_hsd
    result = tukey_hsd(*grupos)
    print("\nComparações par a par:")
    comparacoes = [('A', 'B'), ('A', 'C'), ('B', 'C')]
    for i, (g1, g2) in enumerate(comparacoes):
        diff = abs(grupos[ord(g1)-ord('A')].mean() - grupos[ord(g2)-ord('A')].mean())
        print(f"  {g1} vs {g2}: diferença = {diff:.2f} cm")

# Visualização
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico 1: Boxplot
sns.boxplot(x='Fertilizante', y='Crescimento', data=df, ax=axes[0], palette='Set3')
sns.swarmplot(x='Fertilizante', y='Crescimento', data=df, ax=axes[0], color='black', alpha=0.4, size=4)
axes[0].set_title('Crescimento por Fertilizante', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Crescimento (cm)', fontsize=11)
axes[0].grid(True, alpha=0.3, axis='y')

# Gráfico 2: Médias com IC
medias = df.groupby('Fertilizante')['Crescimento'].mean()
erros = df.groupby('Fertilizante')['Crescimento'].sem() * 1.96

axes[1].bar(medias.index, medias.values, alpha=0.7, color=['#FF6B6B', '#4ECDC4', '#FFE66D'], 
             edgecolor='black', linewidth=1.5)
axes[1].errorbar(medias.index, medias.values, yerr=erros.values, fmt='none', 
                  color='black', capsize=10, capthick=2)
axes[1].set_title('Médias com IC 95%', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Crescimento médio (cm)', fontsize=11)
axes[1].set_xlabel('Fertilizante', fontsize=11)
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 📚 Resumo e Conclusões

### Conceitos-Chave Aprendidos

1. **Teste t para Uma Amostra**:
   - Compara média amostral com valor hipotético
   - Usa distribuição t de Student
   - Apropriado para amostras pequenas (n < 30)
   - Referência: **Montgomery & Runger (2018)**

2. **Teste t para Amostras Independentes**:
   - Compara médias de dois grupos não relacionados
   - Verificar homogeneidade de variâncias (teste de Levene)
   - Calcular tamanho do efeito (d de Cohen)
   - Referência: **Field (2018)**

3. **Teste Qui-Quadrado**:
   - Testa independência entre variáveis categóricas
   - Compara frequências observadas vs esperadas
   - V de Cramér mede força da associação
   - Referência: **Field (2018)**

4. **ANOVA**:
   - Compara médias de 3+ grupos
   - Particiona variância em componentes
   - Testes post-hoc identificam diferenças específicas
   - Referência: **Montgomery & Runger (2018)**

### Boas Práticas

1. **Sempre verificar pressupostos**:
   - Normalidade (teste de Shapiro-Wilk, Q-Q plot)
   - Homogeneidade de variâncias (teste de Levene)
   - Independência das observações

2. **Reportar completamente**:
   - Estatística de teste
   - Valor-p
   - Tamanho do efeito
   - Intervalos de confiança

3. **Interpretar corretamente**:
   - p-valor NÃO é a probabilidade de H₀ ser verdadeira
   - Significância estatística ≠ importância prática
   - Não rejeitar H₀ ≠ aceitar H₀

### Escolha do Teste Apropriado

| Situação | Teste Apropriado |
|----------|------------------|
| Comparar média com valor fixo | Teste t uma amostra |
| Comparar 2 grupos independentes | Teste t independente |
| Comparar 2 grupos pareados | Teste t pareado |
| Comparar 3+ grupos | ANOVA |
| Variáveis categóricas | Qui-quadrado |
| Dados não-normais | Testes não-paramétricos |

### Referências Bibliográficas

**MONTGOMERY, Douglas C.; RUNGER, George C.** *Applied Statistics and Probability for Engineers*. 7. ed. Hoboken: Wiley, 2018.
- Referência completa sobre testes de hipóteses aplicados

**FIELD, Andy.** *Discovering Statistics Using IBM SPSS Statistics*. 5. ed. London: SAGE Publications, 2018.
- Abordagem acessível com exemplos práticos

**COHEN, Jacob.** *Statistical Power Analysis for the Behavioral Sciences*. 2. ed. Hillsdale: Lawrence Erlbaum Associates, 1988.
- Referência definitiva sobre tamanho do efeito e poder estatístico

**WASSERSTEIN, Ronald L.; LAZAR, Nicole A.** The ASA Statement on p-Values: Context, Process, and Purpose. *The American Statistician*, v. 70, n. 2, p. 129-133, 2016.
- Guia oficial sobre interpretação de p-valores

**FISHER, Ronald A.** *Statistical Methods for Research Workers*. Edinburgh: Oliver and Boyd, 1925.
- Obra fundamental que introduziu ANOVA e testes de significância